# TME1

## Import

In [5]:
import numpy as np

## The principal class

In [63]:
class Bandit():
    
    FILENAME = "CTR.txt"
    
    def __init__(self, path=FILENAME):
        """
        Convert the text file into 2 matrix : one for the context and one for the true gain for each example.
        
        :param path: The path of the data file
        
        :type path: String
        """
        with open(path, "r") as file:
            lines = file.read().split("\n")
            
        #Extract data
        lines = lines[:-1]
        contexts = []
        trueGains = []
        for line in lines:
            useless, context, trueGain = line.split(":")
            contexts.append(context.split(";"))
            trueGains.append(trueGain.split(";"))
        self.numData = 1000
        contexts = contexts[:self.numData]#To test with less data
        trueGains = trueGains[:self.numData]#To test with less data
        self.contexts = np.array(contexts, dtype=np.float32)
        self.trueGains = np.array(trueGains, dtype=np.float32)
        self.nb_arms = self.trueGains.shape[1]
        
        #Prepare usefull variable for statistics
        self.averageGains = self.trueGains.sum(axis=0)#compute sum for each columns wich is the global gain for each arm
        
            
    
    def start(self, funcChoice):
        """
        To start playing using the function given.
        
        :param funcChoice: The function to use for the choice
        
        :type funcChoice: Bandit.func()
        
        :return: A 2-uplet wich the optimal regret and the pseudo-optimal regret
        :rtype: (float, float)
        """
        self.gains = np.zeros(self.trueGains.shape[1], dtype=np.float32) #initialise the gains for each arms to 0
        self.nbs = np.zeros(self.trueGains.shape[1], dtype=np.float32) #initialise the number of play for each arms to 0
        self.statsOpti = {"optimal":0.0, 'staticBest':0.0}#to compute gain of optimal strategy and pseudo optimal strategy
        self.time = 0#intialise time to 0
        self.choices = []#to remember choices
        
        
        #param for linUCB algorithm
        
        self.contextDim = self.contexts.shape[1] # context dimension (= 5)
        self.alpha = 0.1 
        self.A = np.zeros((self.trueGains.shape[1], self.contextDim, self.contextDim)) # Aa = Da.T*Da + Id
        self.A[:] = np.identity(self.contextDim) # we initialise A for each arm to an identity matrix (dxd) 
        self.B = np.zeros((self.trueGains.shape[1], self.contextDim)) # ba = Da.T*ca
        self.Teta = np.zeros((self.trueGains.shape[1], self.contextDim)) # matrix of arms features
        
        for trueGain in self.trueGains:
            
            choice = funcChoice()#get the choice with the choosen method
            self.choices.append(choice)
            #update of current gains, number of play, optimal gain and pseudo optimal gain
            self.gains[choice] += self.trueGains[self.time][choice]
            self.nbs[choice] += 1
            self.statsOpti["optimal"] += self.trueGains[self.time][self.optimal()]
            self.statsOpti["staticBest"] += self.trueGains[self.time][self.staticBest()]
            self.time += 1
            
        return self.statsOpti["optimal"]-self.gains.sum(), self.statsOpti["staticBest"]-self.gains.sum()
            
        
            
    def random(self):
        """
        Implementation of random choice
        """
        return np.random.randint(self.trueGains.shape[1])
    
    
    def staticBest(self):
        """
        Implementation of pseudo optimal (also named staticBest)
        """
        return np.argmax(self.averageGains)
    
    
    def optimal(self):
        """
        Implementation of optimal
        """
        return np.argmax(self.trueGains[self.time])
    
    
    def ucb(self):
        """
        Implementation od UCB
        """
        if(self.time<self.nb_arms): #Initialisation to have first estimation
            return self.time
        else:
            return np.argmax((self.gains/self.nbs)+np.sqrt(2*np.log(self.time+1)/self.nbs))
        
        
    def linUCB(self):
        """
        Implementation of linUCB algorithm
        """
        if(self.time<self.nb_arms): #Initialisation to have first estimation
            return self.time
        
        else:
            proba = np.zeros(self.nb_arms)
            context = self.contexts[self.time]
            
            for arm in range(self.nb_arms):
                
                invA = np.linalg.inv(self.A[arm])
                self.Teta[arm] = np.dot(invA, self.B[arm]) # analytic reslut of ridge regression
                proba[arm] = np.dot(self.Teta[arm], context) + self.alpha*np.sqrt(np.dot(np.dot(context.T,invA),context))
            
            choice = np.argmax(proba)
            payoff = self.trueGains[self.time][choice]
            
            # Update choosen arm features
            
            self.A[choice] += np.dot(context,context.T)
            self.B[choice] += payoff*context
            
            return choice
                

## Initialisation and variable printing

In [64]:
b=Bandit()

In [65]:
b.contexts.shape#(number of example; number of arms)

(1000, 5)

In [10]:
print(b.trueGains[:3])#sample to see head data

[[0.10341906 0.19069779 0.         0.10240139 0.03631242 0.07456195
  0.23470241 0.         0.         0.07857373]
 [0.         0.         0.         0.0208271  0.         0.
  0.02258593 0.         0.14654765 0.32459557]
 [0.10957462 0.13662645 0.         0.09841065 0.0747713  0.
  0.         0.         0.01475992 0.19367686]]


## Apply each function choice

In [66]:
b.start(b.random)

(224.5064943432808, 185.38564492203295)

In [67]:
b.start(b.staticBest)

(39.121072590351105, 0.00022316910326480865)

In [68]:
b.start(b.optimal)

(9.357929229736328e-06, -39.12084006331861)

In [69]:
b.start(b.ucb)

(165.64649373292923, 126.52564431168139)

In [70]:
b.start(b.linUCB)

(76.71257954835892, 37.59173012711108)

We can see that as expected:
- ucb is better than random 
- ucb is worse than pseudo-optimal wich is expected (and lin-UCB should be better)
- optimal as an optimal regret of 0 because it is the optimal
- staticBest as an pseudo-optimal of 0 because it is pseudo-optimal
- optimal is better than pseudo-optimal
- optimal regret is always greater than pseudo-optimal regret